importing libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

load the tenserflow dataset

In [2]:
import tensorflow_datasets as tfds

load the data

In [3]:
(train_data, test_data), ds_info = tfds.load('tf_flowers',
                                             split=['train[:80%]', 'train[80%:]'],
                                             shuffle_files=True,
                                             as_supervised=True,
                                             with_info=True)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


Preprocess the data

In [4]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

def preprocess(image, label):
    image = tf.image.resize(image, IMG_SIZE)
    image = image / 255.0
    return image, label

In [5]:
train_data = train_data.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_data = test_data.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

loading pre trained models ResNet50

In [6]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step




-freezing the base model to retained its learned weights



In [7]:
base_model.trainable = False



-adding custome layers on top of the pretrained model



In [8]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(ds_info.features['label'].num_classes, activation='softmax')(x)



-building the model



In [9]:
model = Model(inputs=base_model.input, outputs=predictions)

compile the model

In [10]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

train the model

In [11]:
history = model.fit(train_data,
                    validation_data=test_data,
                    epochs=30)

Epoch 1/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 42s 280ms/step - accuracy: 0.2666 - loss: 1.7488 - val_accuracy: 0.3583 - val_loss: 1.5575
Epoch 2/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 11s 113ms/step - accuracy: 0.3848 - loss: 1.4811 - val_accuracy: 0.4060 - val_loss: 1.4789
Epoch 3/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 10s 112ms/step - accuracy: 0.4044 - loss: 1.4352 - val_accuracy: 0.4264 - val_loss: 1.4071
Epoch 4/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 20s 110ms/step - accuracy: 0.4251 - loss: 1.3941 - val_accuracy: 0.4537 - val_loss: 1.3610
Epoch 5/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 21s 118ms/step - accuracy: 0.4462 - loss: 1.3648 - val_accuracy: 0.4591 - val_loss: 1.3545
Epoch 6/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 19s 105ms/step - accuracy: 0.4552 - loss: 1.3351 - val_accuracy: 0.4169 - val_loss: 1.3733
Epoch 7/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 11s 113ms/step - accuracy: 0.4621 - loss: 1.3148 - val_accuracy: 0.3951 - val_loss: 1.3820
Epoch 8/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 11s 118ms/step - accuracy: 0.4674 - loss: 1.2929 - val_accu

fine tunning dense layers



-unfreeze the layers for fine-tunning



In [12]:
base_model.trainable = True



-recompile the model with lower running rate for fine tunning



In [13]:
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])



-train the model again for fine-tunning



In [14]:
fine_tune_history = model.fit(train_data,
                              validation_data=test_data,
                              epochs=15)

Epoch 1/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 118s 656ms/step - accuracy: 0.3560 - loss: 22.5409 - val_accuracy: 0.2003 - val_loss: 13.2972
Epoch 2/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 29s 314ms/step - accuracy: 0.8301 - loss: 1.8493 - val_accuracy: 0.2153 - val_loss: 22.0707
Epoch 3/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 42s 321ms/step - accuracy: 0.9460 - loss: 0.4129 - val_accuracy: 0.2330 - val_loss: 22.8326
Epoch 4/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 40s 313ms/step - accuracy: 0.9864 - loss: 0.0654 - val_accuracy: 0.2371 - val_loss: 23.8691
Epoch 5/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 29s 314ms/step - accuracy: 0.9986 - loss: 0.0104 - val_accuracy: 0.2629 - val_loss: 21.4786
Epoch 6/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 41s 317ms/step - accuracy: 0.9985 - loss: 0.0074 - val_accuracy: 0.3583 - val_loss: 16.5440
Epoch 7/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 41s 323ms/step - accuracy: 0.9974 - loss: 0.0071 - val_accuracy: 0.4401 - val_loss: 12.8210
Epoch 8/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 29s 319ms/step - accuracy: 1.0000 - loss: 4.2716e-

evaluate the model

In [15]:
test_loss, test_acc = model.evaluate(test_data)

23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 94ms/step - accuracy: 0.7695 - loss: 3.2232




-get predictions



In [16]:
y_true = np.concatenate([y for x, y in test_data], axis=0)
y_pred = np.argmax(model.predict(test_data), axis=1)

23/23 ━━━━━━━━━━━━━━━━━━━━ 8s 228ms/step


classification report

In [17]:
print(classification_report(y_true, y_pred, target_names=ds_info.features['label'].names))

              precision    recall  f1-score   support

   dandelion       0.84      0.84      0.84       159
       daisy       0.71      0.75      0.73       120
      tulips       0.78      0.74      0.76       186
  sunflowers       0.86      0.74      0.79       138
       roses       0.68      0.79      0.73       131

    accuracy                           0.77       734
   macro avg       0.77      0.77      0.77       734
weighted avg       0.78      0.77      0.77       734

